In [8]:
## construct references
annotations = {'human': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/reference/gencode.v27.annotation.no_PAR.gtf',
              'orangutan': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/Susie_Orangutan.filtered.gtf',
              'chimp': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/Clint_Chimp.filtered.gtf',
              'gorilla': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/Susie_Gorilla.filtered.gtf',
              'ponAbe2': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/consensus_gene_set/Orangutan.filtered.gtf',
              'panTro4': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/consensus_gene_set/Chimp.filtered.gtf',
              'gorGor4': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/consensus_gene_set/Gorilla.filtered.gtf'}

fastas = {'human': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/genome_files/Human.fa',
         'orangutan': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/genome_files/Susie_Orangutan.fa',
         'chimp': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/genome_files/Clint_Chimp.fa',
         'gorilla': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/genome_files/Susie_Gorilla.fa',
         'ponAbe2': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/genome_files/Orangutan.fa',
         'panTro4': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/genome_files/Chimp.fa',
         'gorGor4': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/genome_files/Gorilla.fa'}

base_cmd = 'rsem-prepare-reference --star -p 32 --gtf {0} {1} {2}/{2}\n'
with open('cmds.txt', 'w') as outf:
    for g, a in annotations.iteritems():
        f = fastas[g]
        !mkdir -p {g}
        outf.write(base_cmd.format(a, f, g))

In [ ]:
%%bash
cmd="cd ${PWD} && para make -cpu=32 -ram=256g cmds.txt"
ssh ku $cmd

In [27]:
import os
rnaseq_dir = '/hive/groups/recon/projs/primates/more_rnaseq_data/iPSC-data'
base_cmd = 'rsem-calculate-expression -p 32 --star --paired-end {0} {1} {2}/{2} {2}\n'
with open('cmds.txt', 'w') as outf:
    for g in ['human', 'orangutan', 'chimp', 'gorilla']:
        fwd = os.path.join(rnaseq_dir, g + '.fwd.fq')
        rev = os.path.join(rnaseq_dir, g + '.rev.fq')
        assert os.path.exists(fwd)
        assert os.path.exists(rev)
        outf.write(base_cmd.format(fwd, rev, g))
    # add in the original genome CAT sets
    for g, ref in [['chimp', 'panTro4'], ['gorilla', 'gorGor4'], ['orangutan', 'ponAbe2']]:
        fwd = os.path.join(rnaseq_dir, g + '.fwd.fq')
        rev = os.path.join(rnaseq_dir, g + '.rev.fq')
        assert os.path.exists(fwd)
        assert os.path.exists(rev)
        outf.write(base_cmd.format(fwd, rev, ref))
    # add in the ensembl sets
    for g, ref in [['chimp', 'pantro_ensembl'], ['gorilla', 'gorgor3_ensembl'], ['orangutan', 'ponabe_ensembl']]:
        fwd = os.path.join(rnaseq_dir, g + '.fwd.fq')
        rev = os.path.join(rnaseq_dir, g + '.rev.fq')
        assert os.path.exists(fwd)
        assert os.path.exists(rev)
        outf.write(base_cmd.format(fwd, rev, ref))

In [33]:
# hack to finish EM step, can't run on cluster
import os
base_cmd = 'rsem-calculate-expression --alignments -p 4 --paired-end {0}.temp/{0}.bam {0}/{0} {0}\n'
with open('cmds_tmp.txt', 'w') as outf:
    for g in ['human', 'orangutan', 'chimp', 'gorilla', 'panTro4', 'ponAbe2', 'pantro_ensembl',
             'gorgor3_ensembl', 'ponabe_ensembl', 'gorGor4']:
        outf.write(base_cmd.format(g))

In [28]:
%%bash
cmd="cd ${PWD} && para make -cpu=32 -ram=256g cmds.txt"
ssh ku $cmd

Process is interrupted.


In [18]:
# load dfs
import pandas as pd

base_dir = '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/'
dfs = {'chimp': pd.read_csv(os.path.join(base_dir, 'Clint_Chimp.fixed.filtered.gp_info'), sep='\t'),
      'oranguatan': pd.read_csv(os.path.join(base_dir, 'Susie_Orangutan.fixed.filtered.gp_info'), sep='\t'),
      'gorilla': pd.read_csv(os.path.join(base_dir, 'Susie_Gorilla.fixed.filtered.gp_info'), sep='\t')}

base_dir = '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/consensus_gene_set/'
for g, og in [['panTro4', 'Chimp'], ['ponAbe2', 'Orangutan'], ['gorGor4', 'Gorilla']]:
    dfs[g] = pd.read_csv(os.path.join(base_dir, og + '.filtered.gp_info'), sep='\t')

from tools.sqlInterface import *
dfs['human'] = load_annotation('/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/databases/Human.db')
dfs['human'].columns = ['transcript_id', 'gene_id', 'transcript_name', 'gene_name', 'gene_biotype', 'transcript_biotype']

In [185]:
# load ensembl annotations
from tools.misc import *
from glob import glob
def load_map(gtf):
    r = []
    for l in open(gtf):
        l = l.rstrip().split('\t')
        x = parse_gtf_attr_line(l[-1])
        if 'gene_name' not in x:
            continue
        r.append([x['gene_id'], x['gene_name'], x['gene_biotype']])
    m = pd.DataFrame(r, columns=['gene_id', 'gene_name', 'gene_biotype'])
    m = m.drop_duplicates()
    return m


for ref in ['pantro_ensembl', 'gorgor3_ensembl', 'ponabe_ensembl']:
    gtf = glob(ref + '/*gtf')
    assert len(gtf) == 1
    gtf = gtf[0]
    df = load_map(gtf)
    dfs[ref] = df

In [53]:
# load expression
exp_dfs = {}
for g in dfs:
    if 'ponAbe' in g or 'ponabe' in g or 'oranguatan' in g:
        continue
    exp_dfs[g] = pd.read_csv('{}.genes.results'.format(g), header=0, sep='\t')

In [212]:
# merge dfs
merged_dfs = {}
for g, exp_df in exp_dfs.iteritems():
    if g == 'human' or 'ensembl' in g:
        m = exp_df.merge(dfs[g][['gene_id', 'gene_name', 'gene_biotype']].drop_duplicates(), on='gene_id')
        m = m[m.gene_biotype == 'protein_coding']
        m = m[['gene_name', 'TPM']]
    else:
        m = exp_df.merge(dfs[g][['gene_id', 'source_gene_common_name', 'gene_biotype']].drop_duplicates(), on='gene_id')
        m = m[m.gene_biotype == 'protein_coding']
        m = m[['source_gene_common_name', 'TPM']]
        m.columns = ['gene_name', 'TPM']
    m = m.groupby('gene_name').aggregate(sum)
    merged_dfs[g] = m.reset_index()
    print g, m.TPM.sum()

gorGor4 824882.46
panTro4 805766.72
chimp 573281.86
gorilla 681951.36
human 605480.73
gorgor3_ensembl 689211.39
pantro_ensembl 708656.08


In [213]:
# fix names
merged_dfs['human'].columns = ['gene_name', 'Human (GENCODE V27)']
merged_dfs['chimp'].columns = ['gene_name', 'Clint/Chimpanzee (CAT)']
merged_dfs['gorilla'].columns = ['gene_name', 'Susie/Gorilla (CAT)']
#merged_dfs['orangutan'].columns = ['gene_name', 'Susie/Orangutan (CAT)']
merged_dfs['pantro_ensembl'].columns = ['gene_name', 'panTro4 (Ensembl V90)']
#merged_dfs['ponabe_ensembl'].columns = ['gene_name', 'ponAbe2 (Ensembl V90)']
merged_dfs['gorgor3_ensembl'].columns = ['gene_name', 'gorGor3 (Ensembl V90)']
merged_dfs['panTro4'].columns = ['gene_name', 'panTro4 (CAT)']
#merged_dfs['ponAbe2'].columns = ['gene_name', 'ponAbe2 (CAT)']
merged_dfs['gorGor4'].columns = ['gene_name', 'gorGor4 (CAT)']

In [214]:
# now we merge and group
combined_dfs = {}
combined_dfs['chimp'] = merged_dfs['human'].merge(merged_dfs['chimp'], how='outer', on='gene_name').merge(merged_dfs['panTro4'], how='outer', on='gene_name').merge(merged_dfs['pantro_ensembl'], how='outer', on='gene_name')
combined_dfs['gorilla'] = merged_dfs['human'].merge(merged_dfs['gorilla'], how='outer', on='gene_name').merge(merged_dfs['gorGor4'], how='outer', on='gene_name').merge(merged_dfs['gorgor3_ensembl'], how='outer', on='gene_name')
#combined_dfs['orangutan'] = merged_dfs['human'].merge(merged_dfs['orangutan'], how='outer', on='gene_name').merge(merged_dfs['ponAbe2'], how='outer', on='gene_name').merge(merged_dfs['ponabe_ensembl'], how='outer', on='gene_name')

melted_dfs = {}
for g, df in combined_dfs.iteritems():
    df = pd.melt(df, id_vars=['gene_name', 'Human (GENCODE V27)'])
    df.columns = ['gene_name', 'Human (GENCODE V27)', 'assembly/annotation', 'TPM']
    melted_dfs[g] = df

In [215]:
def plot_fn(pdf, df, cols, num_human, num_cat, num_old_cat, num_ensembl):
    palette = sns.color_palette()
    #palette = [palette[3], palette[2], palette[1]]
    palette = palette[1:]
    with open(pdf, 'w') as outf, PdfPages(outf) as pdf:
        g = sns.lmplot(data=df, x='Human (GENCODE V27)', y='TPM', col='assembly/annotation',
                      hue='assembly/annotation', fit_reg=False,
                      scatter_kws={'marker': '+', 'alpha': 0.5, 's': 3,  'rasterized': True},
                      col_order=cols,
                      palette=palette)
        x = np.linspace(0, 250)
        for col, ax in zip(*[cols, g.axes[0]]):
            tmp = df[df['assembly/annotation'] == col]
            tmp = tmp[(tmp['Human (GENCODE V27)'].notnull()) & (tmp['TPM'].notnull())]
            c, p = pearsonr(tmp['Human (GENCODE V27)'], tmp['TPM'])
            ax.set_title(ax.get_title() + '\nPearson r={:.2f} p={:.2f}'.format(c, p))
            ax.set_xlim(0, 250)
            ax.set_ylim(0, 250)
            ax.plot(x, x, 'r--', color='black', alpha=0.7, linewidth=1)
        multipage_close(pdf, False)
        g = sns.barplot(x=['Human (GENCODE V27)'] + cols,
                   y=[num_human, num_cat, num_old_cat, num_ensembl])
        g.set_title('Number of genes found in RNA-seq')
        sns.despine()
        plt.xticks(rotation=70)
        plt.title('Number of genes with non-zero expression')
        multipage_close(pdf, False)
        

import seaborn as sns
from cat.plots import *
from scipy.stats import *
hdf =  merged_dfs['human']
num_human = len(hdf[hdf['Human (GENCODE V27)'] > 0.1])
for g, df in melted_dfs.iteritems():
    pdf = g + '.pdf'
    if g == 'chimp':
        cols = ['Clint/Chimpanzee (CAT)', 'panTro4 (CAT)', 'panTro4 (Ensembl V90)']
    elif g == 'gorilla':
        cols = ['Susie/Gorilla (CAT)', 'gorGor4 (CAT)', 'gorGor3 (Ensembl V90)']
    else:
        cols = ['Susie/Orangutan (CAT)', 'ponAbe2 (CAT)', 'ponAbe2 (Ensembl V90)']
    if g == 'chimp':
        num_cat = len(merged_dfs['chimp'][merged_dfs['chimp']['Clint/Chimpanzee (CAT)'] > 0.1])
        num_old_cat = len(merged_dfs['panTro4'][merged_dfs['panTro4']['panTro4 (CAT)'] > 0.1])
        num_ensembl = len(merged_dfs['pantro_ensembl'][merged_dfs['pantro_ensembl']['panTro4 (Ensembl V90)'] > 0.1])
    elif g == 'gorilla':
        num_cat = len(merged_dfs['gorilla'][merged_dfs['gorilla']['Susie/Gorilla (CAT)'] > 0.1])
        num_old_cat = len(merged_dfs['gorGor4'][merged_dfs['gorGor4']['gorGor4 (CAT)'] > 0.1])
        num_ensembl = len(merged_dfs['gorgor3_ensembl'][merged_dfs['gorgor3_ensembl']['gorGor3 (Ensembl V90)'] > 0.1])
    else:
        num_cat = len(merged_dfs['orangutan'][merged_dfs['orangutan']['Susie/Orangutan (CAT)'] > 0.1])
        num_old_cat = len(merged_dfs['ponAbe2'][merged_dfs['ponAbe2']['ponAbe2 (CAT)'] > 0.1])
        num_ensembl = len(merged_dfs['ponabe_ensembl'][merged_dfs['ponabe_ensembl']['ponAbe2 (Ensembl V90)'] > 0.1])
    plot_fn(pdf, df, cols, num_human, num_cat, num_old_cat, num_ensembl)